In [1]:
# Imports
import os
import shutil
from glob import glob
from zipfile import ZipFile

from osgeo import gdal
from osgeo import ogr
import geopandas as gpd
import pandas as pd
import numpy as np
import rioxarray as rxr

from shapely.geometry import MultiPolygon
from geocube.vector import vectorize

## Inputs

### Input Data

In [2]:
# Input Files

ms_buildings_tiles: 'CWLFilePathInput' = "data/MS_bldgs_all_tiles.json"

# Zipped inputs
critical_habitats_zip: 'CWLFilePathInput' = "data/critical_habitats.zip"
flamelen_zip: 'CWLFilePathInput' = "data/flamelen.zip"
toa_zip: 'CWLFilePathInput' = "data/toa.zip"

### Input Parameters

In [3]:
# Input Parameters
# budget: 'CWLIntInput' = 5

## Outputs

### Intermediate Data

### Output Data

In [4]:
# Output Files
burned_area_zip: 'CWLFilePathOutput' = "burned_area.zip"
building_damage_zip: 'CWLFilePathOutput' = "building_damage.zip"
habitat_damage_zip: 'CWLFilePathOutput' = "habitat_damage.zip"
damage_response_zip: 'CWLFilePathOutput' = "damage_response.zip"
footprints_polygons: 'CWLFilePathOutput' = "footprints_polygons.json"

## Code

In [5]:
critical_habitats_dir = "critical_habitats"
flamelen_dir = "flamelen"
toa_dir = "toa"

burned_area_dir = "burned_area"
building_damage_dir = "building_damage"
habitat_damage_dir = "habitat_damage"
damage_response_dir = "damage_response"
tmp_dir = "tmp"

In [6]:
def unzip(zipfile, todir):
    if not os.path.exists(todir):
        with ZipFile(zipfile) as zip_file:
            filenames = zip_file.namelist()
            zip_maindir = None
            if filenames[0].endswith("/"):
                zip_maindir = filenames[0]
            
            for filename in zip_file.namelist():
                if filename.startswith("__"):
                    continue
                if zip_maindir and not filename.startswith(zip_maindir):
                    continue

                tofilename = filename
                if zip_maindir:
                    tofilename = filename.replace(zip_maindir, todir + "/", 1)

                if tofilename.endswith("/"):
                    os.makedirs(tofilename)
                else:
                    source = zip_file.open(filename)
                    target = open(tofilename, "wb")
                    with source, target:
                        shutil.copyfileobj(source, target)

### Helper Functions

In [7]:
def _make_polygons(filename, in_path, out_path, prefix):
    """
    Function takes fire footprint raster, converts the intensity values to binary values (burn/not burn) and writes new raster.
    It also creates a geopandas dataframe of footprint polygons

    Parameters
    ----------
    filename (str)
    in_path (str)
    out_path (str)
    prefix (str) ex. burned_area

    Returns
    -------
    geopandas dataframe with footprint polygon
    """

    #open tif with gdal and convert to array
    print(f'input raster: {filename}')
    ds = gdal.Open(in_path+ '/' + filename)
    gt = ds.GetGeoTransform()
    proj = ds.GetProjection()
    band = ds.GetRasterBand(1)
    array = band.ReadAsArray()
    
    # create burn/not burn binary mask

    binmask = np.where((array > 0),1,0)  # keep all the values that are greater than 0

    # export
    driver = gdal.GetDriverByName("GTiff")
    driver.Register()
    
    bin_filename = f"{prefix}-{filename[4:]}"
    print(f'output raster: {bin_filename}')
    outds = driver.Create(out_path+ '/' + bin_filename, xsize = binmask.shape[1],
                      ysize = binmask.shape[0], bands = 1, 
                      eType = gdal.GDT_Int16)
    outds.SetGeoTransform(gt)
    outds.SetProjection(proj)
    outband = outds.GetRasterBand(1)
    outband.WriteArray(binmask)
    outband.SetNoDataValue(np.nan)
    outband.FlushCache()

    # close your datasets and bands!
    outband = None
    outds = None
    
    #open bin_mask and polygonize it
    bin_ = rxr.open_rasterio(out_path+ '/' + bin_filename).squeeze('band', drop=True)

    polygons = vectorize(bin_)
    polygons.rename(columns={None: "value"},inplace=True)
    
    # polygons = polygonize(bin_, gt)
    
    perimeter = polygons[polygons['value']==1.0]  # select outside polygon
    
    # returns polygon in geodataframe
    perimeter['filename'] = filename
    return perimeter

In [8]:
def _make_damage_response_rasters(filename,in_path,out_path, prefix):
    """
    This script takes fire footprint raster files for flame length and maps the values to 
    building damage response values (0, 25, 40, 55, 70, 85, 100) and writes a new raster

    inputs: 
    filename (str)
    in_path (str)
    out_path (str)
    prefix (str)

    outputs:
    damage response rasters
    """
   
    #open tif with gdal and convert to array
    ds = gdal.Open(in_path+ '/' + filename)
    gt = ds.GetGeoTransform()
    proj = ds.GetProjection()
    band = ds.GetRasterBand(1)
    array = band.ReadAsArray()
    
    # map flame length to building damage response values
    # approach as per the Wildfire Risk to Communities paper, https://www.fs.usda.gov/rds/archive/Catalog/RDS-2020-0016
    
    #<10' flame --> 0 building response function value
    #>=10' flame --> 100
    
    # New modified binary threshold for calculating building damage rasters
    array = np. where((array < 10),0, array)
    array = np.where((array >= 10), 100, array)
    
    #binmask = np.where((array > 0),1,0)  # keep all the values that are greater than 0

    # export
    driver = gdal.GetDriverByName("GTiff")
    driver.Register()
    
    new_filename = f"{prefix}-{filename[-22:]}"
    #bin_filename = f"damage_response-{filename[9:]}"
    print(new_filename)
    
    #out_path='../test/'+new_filename
    outds = driver.Create(out_path+ '/' + new_filename, xsize = array.shape[1],
                      ysize = array.shape[0], bands = 1, 
                      eType = gdal.GDT_Int16)
    outds.SetGeoTransform(gt)
    outds.SetProjection(proj)
    outband = outds.GetRasterBand(1)
    outband.WriteArray(array)
    outband.SetNoDataValue(np.nan)
    outband.FlushCache()

    # close your datasets and bands!
    outband = None
    outds = None

    pass

SyntaxError: invalid character '‹' (U+2039) (3760461208.py, line 30)

In [ ]:
def _groupby_multipoly(df, by, aggfunc="first"):
    """
    This function make multipolygons from polygons for fire footprints

    inputs:
    geopandas dataframe with polygons

    outputs:
    geopandas dataframe with multipolygons
    """
    
    data = df.drop(labels=df.geometry.name, axis=1)
    aggregated_data = data.groupby(by=by).agg(aggfunc)

    # Process spatial component
    def merge_geometries(block):
        return MultiPolygon(block.values)

    g = df.groupby(by=by, group_keys=False)[df.geometry.name].agg(
        merge_geometries
    )

    # Aggregate
    aggregated_geometry = gpd.GeoDataFrame(g, geometry=df.geometry.name, crs=df.crs)
    # Recombine
    aggregated = aggregated_geometry.join(aggregated_data)
    return aggregated

In [ ]:
def _make_bldg_damage_raster(filename,shp_filename,in_path,out_path, prefix,out_path_tmp):

    """
    This function takes in damage response rasters and creates binary building 
    damage (tmp) rasters and building damage rasters with varying intensity.

    inputs:
    
    filename (str)
    shp_filename (str)
    in_path (str)
    out_path (str)
    prefix (str)
    out_path_tmp (str)

    outputs:

    raster files
    """
    
    fn_ras = in_path+ '/' + 'damage_response-'+filename
    print('input raster: ',fn_ras) 
    ras_ds = gdal.Open(fn_ras)
    intensity_array = ras_ds.GetRasterBand(1).ReadAsArray()
    driver = ogr.GetDriverByName('ESRI Shapefile')
    vec_ds = driver.Open(shp_filename)
    lyr = vec_ds.GetLayer() 
    geot = ras_ds.GetGeoTransform()
    geo_proj = ras_ds.GetProjection() 
    
    # Setup the New Raster
    drv_tiff = gdal.GetDriverByName("GTiff") 
    out_net=f'{out_path_tmp+ "/" + prefix}-binary-{filename}'
    print(out_net)
    chn_ras_ds = drv_tiff.Create(out_net, ras_ds.RasterXSize, ras_ds.RasterYSize, 1, gdal.GDT_Float32)
    chn_ras_ds.SetGeoTransform(geot)
    chn_ras_ds.SetProjection(geo_proj) 
    gdal.RasterizeLayer(chn_ras_ds, [1], lyr, burn_values=[1])
    chn_ras_ds.GetRasterBand(1).SetNoDataValue(np.nan) 
    chn_ras_ds = None
    
    # open binary bldg damage raster back up
    ds = gdal.Open(out_net)
    gt = ds.GetGeoTransform()
    proj = ds.GetProjection()
    band = ds.GetRasterBand(1)
    bin_array = band.ReadAsArray()
    
    # get array and use as mask to get values from damage_response raster array
    damage_array = intensity_array*bin_array.astype(int)
    damage_array = np.where((damage_array > 0), 1, 0) # keep all the values that are greater than 0
    
    #print(damage_array.shape)
    
    # write new raster using chn_ras_ds.GetRasterBand(1).WriteArray(binmask)
    # export
    driver_ = gdal.GetDriverByName("GTiff")
    driver_.Register()
    out=f'{out_path+ "/" + prefix}-{filename}'
    print(f'output raster: {out}')
    outds = driver_.Create(out, xsize = damage_array.shape[1],
                      ysize = damage_array.shape[0], bands = 1, 
                      eType = gdal.GDT_Int16)
    outds.SetGeoTransform(gt)
    outds.SetProjection(proj)
    outband = outds.GetRasterBand(1)
    outband.WriteArray(damage_array)
    outband.SetNoDataValue(np.nan)
    outband.FlushCache()

    # close datasets and bands
    outband = None
    outds = None
    
    pass

In [ ]:
def _make_raster(filename, shp_filename, in_path, out_path, prefix):
    """
    This function writes raster files

    parameters:
    
        filename (str)
        shp_filename (str)
        in_path (str)
        out_path (str)
        prefix (str)

    outputs:

        raster files
    """
    
    fn_ras = in_path+'/' + 'toa-'+filename
    ras_ds = gdal.Open(fn_ras)
    driver = ogr.GetDriverByName('ESRI Shapefile')
    vec_ds = driver.Open(shp_filename) 
    lyr = vec_ds.GetLayer() 
    geot = ras_ds.GetGeoTransform()
    geo_proj = ras_ds.GetProjection() 
    
    # Setup the New Raster
    drv_tiff = gdal.GetDriverByName("GTiff") 
    out_net=out_path+ '/' + prefix+filename
    print('writing output raster: ',out_net)
    chn_ras_ds = drv_tiff.Create(out_net, ras_ds.RasterXSize, ras_ds.RasterYSize, 1, gdal.GDT_Float32)
    chn_ras_ds.SetGeoTransform(geot)
    chn_ras_ds.SetProjection(geo_proj) 
    gdal.RasterizeLayer(chn_ras_ds, [1], lyr, burn_values=[1])
    chn_ras_ds.GetRasterBand(1).SetNoDataValue(np.nan) 
    chn_ras_ds = None
    pass

### Main Functions

In [ ]:
def make_burned_area_rasters(toa_dir, out_burned_area_dir, footprints_polygons_path):
    prefix='burned_area'

    files = [file for file in os.listdir(toa_dir) if file.endswith(".tif")]
    gdf = gpd.GeoDataFrame(columns=['feature'],geometry='feature',crs='EPSG:32610')
    for file in files:
        new_gdf = _make_polygons(file, toa_dir, out_burned_area_dir, prefix)
        gdf = pd.concat([gdf, new_gdf], ignore_index=True)
    gdf = gdf.drop(['feature'], axis=1)
    gdf = gdf.set_geometry("geometry")
    gdf.to_file(footprints_polygons_path, driver="GeoJSON")   

In [ ]:
def make_damage_response_rasters(flamelen_dir, out_damage_response_dir):
    prefix='damage_response'
    files = [file for file in os.listdir(flamelen_dir) if file.endswith(".tif")]
    for file in files:
        _make_damage_response_rasters(file, flamelen_dir, out_damage_response_dir, prefix)

In [ ]:
def make_building_damage_rasters(damage_response_dir, ms_buildings_tiles_path, footprints_polygons_path, tmp_dir, out_building_damage_dir):
    # footprint_polygons

    footprint_polygons = gpd.read_file(footprints_polygons_path)
    footprint_polygons = footprint_polygons.set_crs('EPSG:32610',allow_override=True)

    # make multipolygons from polygons for fire footprints
    grouped = _groupby_multipoly(footprint_polygons, by='filename').reset_index()

    # load bldgs
    bldgs = gpd.read_file(ms_buildings_tiles_path).to_crs('EPSG:32610') 

    # intersect fire footprint MPs with bldgs polygons
    intersection = gpd.overlay(grouped, bldgs, how='intersection')

    # for given fire footprint, merge (essentially a groupby) bldg polygons into multipolygon
    gdf = intersection.dissolve(by='filename').reset_index()

    prefix='building_damage'

    for i in range(len(gdf)):
        #create shp files
        shp_filename=f"{tmp_dir + '/' + prefix}-{gdf.iloc[i].filename[4:22]}.shp"
        gdf.iloc[[i]].to_file(driver = 'ESRI Shapefile', filename=shp_filename)
        
        root_filename=f"{gdf.iloc[i].filename[4:22]}.tif"
        #print(root_filename)
        _make_bldg_damage_raster(root_filename,shp_filename,damage_response_dir,out_building_damage_dir, prefix, tmp_dir)  

In [ ]:
def make_habitat_damage_raster(toa_dir, critical_habitats_dir, footprints_polygons_path, tmp_dir, out_habitat_damage_dir):
    # footprint_polygons
    footprint_polygons = gpd.read_file(footprints_polygons_path)
    footprint_polygons = footprint_polygons.set_crs('EPSG:32610',allow_override=True)

    # make multipolygons from polygons for fire footprints
    grouped = _groupby_multipoly(footprint_polygons, by='filename').reset_index()
  
    # load critical habitat
    habitat_shape_file = glob(os.path.join(critical_habitats_dir, '*.shp'))[0]
    habitat = gpd.read_file(habitat_shape_file).to_crs('EPSG:32610') 

    # intersect fire footprint MPs with habitat polygons
    intersection = gpd.overlay(grouped, habitat, how='intersection')

    # for given fire footprint, merge (essentially a groupby) bldg polygons into multipolygon
    gdf = intersection.dissolve(by='filename').reset_index()

    prefix='habitat_damage-'

    for i in range(len(gdf)):
        #create shp files
        shp_filename=f"{tmp_dir + '/' + prefix}-{gdf.iloc[i].filename[4:22]}.shp"
        gdf.iloc[[i]].to_file(driver = 'ESRI Shapefile', filename=shp_filename)
        
        root_filename=f"{gdf.iloc[i].filename[4:22]}.tif"
        #print(root_filename)
        _make_raster(root_filename,shp_filename,toa_dir,out_habitat_damage_dir, prefix)

## Main Code

In [ ]:
# Extract zip files
unzip(critical_habitats_zip, critical_habitats_dir)
unzip(flamelen_zip, flamelen_dir)
unzip(toa_zip, toa_dir)

In [ ]:
# Make output directories
os.makedirs(burned_area_dir, exist_ok=True)
os.makedirs(building_damage_dir, exist_ok=True)
os.makedirs(habitat_damage_dir, exist_ok=True)
os.makedirs(damage_response_dir, exist_ok=True)
os.makedirs(tmp_dir, exist_ok=True)

In [ ]:
make_burned_area_rasters(toa_dir, burned_area_dir, footprints_polygons)

input raster: toa-24_31-2925-0000005.tif
output raster: burned_area-24_31-2925-0000005.tif
input raster: toa-24_31-2925-0000004.tif
output raster: burned_area-24_31-2925-0000004.tif
input raster: toa-24_31-2925-0000006.tif
output raster: burned_area-24_31-2925-0000006.tif
input raster: toa-24_31-2925-0000007.tif
output raster: burned_area-24_31-2925-0000007.tif


/Users/varun/miniforge3/envs/landoptmet/lib/python3.10/site-packages/geopandas/geodataframe.py:1456: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)
/Users/varun/miniforge3/envs/landoptmet/lib/python3.10/site-packages/geopandas/geodataframe.py:1456: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)
/Users/varun/miniforge3/envs/landoptmet/lib/python3.10/site-packages/geopandas/geodataframe.py:1456: SettingWithCopyWarning: 
A value is trying to be set 

In [ ]:
make_damage_response_rasters(flamelen_dir, damage_response_dir)
make_building_damage_rasters(damage_response_dir, ms_buildings_tiles, footprints_polygons, tmp_dir, building_damage_dir)

damage_response-24_31-2925-0000006.tif
damage_response-24_31-2925-0000007.tif
damage_response-24_31-2925-0000005.tif
damage_response-24_31-2925-0000004.tif
                       filename  value     index  release  \
0    toa-24_31-2925-0000004.tif    1.0   5253492        2   
1    toa-24_31-2925-0000004.tif    1.0   7300005        2   
2    toa-24_31-2925-0000004.tif    1.0   8158323        2   
3    toa-24_31-2925-0000004.tif    1.0   8158438        2   
4    toa-24_31-2925-0000004.tif    1.0   8674150        2   
..                          ...    ...       ...      ...   
193  toa-24_31-2925-0000007.tif    1.0  10709544        2   
194  toa-24_31-2925-0000007.tif    1.0  10709658        2   
195  toa-24_31-2925-0000007.tif    1.0  10748806        2   
196  toa-24_31-2925-0000007.tif    1.0  11153576        2   
197  toa-24_31-2925-0000007.tif    1.0  11456832        2   

                                              geometry  
0    POLYGON ((786366.011 4151660.290, 786365.803 4...

In [ ]:
make_habitat_damage_raster(toa_dir, critical_habitats_dir, footprints_polygons, tmp_dir, habitat_damage_dir)

writing output raster:  habitat_damage/habitat_damage-24_31-2925-0000004.tif


In [ ]:
# Create zip files
shutil.make_archive(burned_area_dir, 'zip', burned_area_dir)
os.rename(f"{burned_area_dir}.zip", burned_area_zip)

shutil.make_archive(building_damage_dir, 'zip', building_damage_dir)
os.rename(f"{building_damage_dir}.zip", building_damage_zip)

shutil.make_archive(damage_response_dir, 'zip', damage_response_dir)
os.rename(f"{damage_response_dir}.zip", damage_response_zip)

shutil.make_archive(habitat_damage_dir, 'zip', habitat_damage_dir)
os.rename(f"{habitat_damage_dir}.zip", habitat_damage_zip)

In [ ]:
# Clean up
shutil.rmtree(critical_habitats_dir)
shutil.rmtree(flamelen_dir)
shutil.rmtree(toa_dir)
shutil.rmtree(burned_area_dir)
shutil.rmtree(building_damage_dir)
shutil.rmtree(habitat_damage_dir)
shutil.rmtree(damage_response_dir)
shutil.rmtree(tmp_dir)